## Pré-processamento

In [213]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import string
from sklearn.model_selection import train_test_split
#import spacy

df = pd.read_csv('factnews_dataset.csv')

#criar grupo treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train_df['tokens'] = train_df['sentences'].apply(lambda x: nltk.word_tokenize(str(x).lower()))

# Tokenizar sem stopwords
train_df['tokens_no_stopwords'] = train_df['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])

## Análise exploratória

In [214]:
# Contar número de linhas por classe
contagem_classes = train_df['classe'].value_counts()
print(contagem_classes)


 0    3394
-1    1115
 1     443
Name: classe, dtype: int64


Detetar palavras únicas entre as classes

In [215]:
classes = df['classe'].unique()

def palavras_unicas_por_classe(df, token_col):
    
    # contar tokens por classe
    tokens_por_classe = {}
    for classe in classes:
        linhas_classe = df[df['classe'] == classe][token_col].dropna()
        contagem = {}
        for lista in linhas_classe:
            for token in lista:
                if token in contagem:
                    contagem[token] += 1
                else:
                    contagem[token] = 1
        tokens_por_classe[classe] = contagem

    # detetar palavras exclusivas por classe
    palavras_unicas = {}
    for classe in classes:
        # lista de palavras das outras classes
        outras_palavras = []
        for outra_classe in classes:
            if outra_classe != classe:
                for token in tokens_por_classe[outra_classe]:
                    outras_palavras.append(token)

        # dicionário palavras exclusivas com contagens
        unicas_classe = {}
        for palavra, contagem in tokens_por_classe[classe].items():
            exclusiva = True
            for p in outras_palavras:
                if p == palavra:
                    exclusiva = False
                    break
            if exclusiva:
                unicas_classe[palavra] = contagem

        palavras_unicas[classe] = unicas_classe

    return palavras_unicas

#palavras_unicas_por_classe(train_df,'tokens')


ver a proporção de advérbios e adjetivos entre as classes

In [ ]:
nltk.download('mac_morpho') 
from nltk.corpus import mac_morpho

dic_palavras = {}  # Dic para palavras de cada classe gramatical
for palavra, tag in mac_morpho.tagged_words():  # (palavra, tag)
    if tag not in dic_palavras: 
        dic_palavras[tag] = set()
    dic_palavras[tag].add(palavra.lower())  # associar palavra à respetiva tag

print (dic_palavras)

def proporcao_adverbios_adjetivos_por_classe(df, coluna_tokens):
    classes = df['classe'].unique()  
    proporcoes_por_classe = {}     

    lista_adverbios = dic_palavras.get('ADV', set())  
    lista_adjetivos = dic_palavras.get('ADJ', set())  

    
    for classe in classes: 
        tokens_por_classe = df[df['classe'] == classe][coluna_tokens].dropna()  
            
        total_adverbios_classe = 0   
        total_adjetivos_classe = 0    
        
        # contar advérbios e adjetivos
        for lista_tokens in tokens_por_classe:
            total_adverbios_classe += sum(1 for token in lista_tokens if token in lista_adverbios)
            total_adjetivos_classe += sum(1 for token in lista_tokens if token in lista_adjetivos)
        
        proporcoes_por_classe[classe] = {
            'total_adverbios': total_adverbios_classe,
            'total_adjetivos': total_adjetivos_classe
        }
    
    return proporcoes_por_classe

proporcao_adverbios_adjetivos_por_classe(train_df, 'tokens')

[nltk_data] Downloading package mac_morpho to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


{'N': {'chuteiras', 'aerporto', 'comissão-', 'bateristas', 'catatônicos', 'bacteriazinha', 'traumas', 'poodle', 'discursividade', 'segmento', 'desfolhando', 'derrame', 'frugalidade', '1893', 'pretendente', 'peregrinação', 'vibrião', 'aterros', 'cdf', 'quasares', 'nemestrina', 'bestialização', 'desejância', 'espinosismo', 'receio', 'sacrifícios', 'clã', 'transpiração', 'viaturas', 'prazeres', 'folgas', 'infravermelho', 'narrações', 'moedinhas', 'ícone-figura', 'morosidade', 'banal', 'desentendimento', 'vitrine', 'honorários', 'chiqueirinho', 'agulas', 'cubano', 'cordeiro', 'gole', 'milímetros', 'softwares', 'gás/carvão', 'genes', 'cunha', '58', 'crees', 'serralheria', 'super-herói', 'departametos', 'escuderia', 'cs', 'correspondências', 'bicadas', 'veia', 'incidente', 'trvessa', 'telepata', 'justa', 'pesos', 'molestamento', 'paragens', 'argumentações', 'lábios', 'ns', 'ritualista', 'bolsos', 'caveira', 'porta-aviões', 'firmeza', 'aprendiz', 'base', 'epitalâmio', 'prefeita', 'greve', 'ar

{0: {'total_adverbios': 26007, 'total_adjetivos': 15778},
 -1: {'total_adverbios': 7702, 'total_adjetivos': 4684},
 1: {'total_adverbios': 3964, 'total_adjetivos': 2278}}

## Contruir regras para classificar o texto

Regras para a classe -1 (citação)

In [217]:
def citacao(text):
    return '"' in str(text) or '“' in str(text) or '”' in str(text) or '[' in str(text) 

Regras para a classe 0 (facto)

Regras para a classe 1 (viés)

In [218]:
def bias(text):
    return '?' in str(text)

In [219]:
def classificar(row):
    if citacao(row['sentences']):
        return -1  
    if bias(row['sentences']):
        return 1
    return 0

ver verbo verbo numa frase - prob um bias

## Aplicar classificação no conjunto de teste

In [220]:
test_df['Previsto'] = test_df.apply(classificar, axis=1)


## Avaliar as regras de classificação

In [221]:
print(test_df[['sentences', 'classe', 'Previsto']].head())

                                              sentences  classe  Previsto
996   No primeiro turno, o candidato da esquerda chi...       0         0
468   Esta é a segunda vez em menos de um ano que o ...       0         0
6077  A margem de erro é dois pontos para mais ou pa...       0         0
2127  Passei um dia na cidade nesta semana -moro no ...      -1         0
2080  Na ocasião, o peemedebista discordou dos outro...       0         0


In [222]:
certos = (test_df['classe'] == test_df['Previsto']).sum()
total = len(df)
print(f'Certos: {certos} de {total}')
print(f'Accuracy: {certos/total:.2%}')

for c in classes:
    linhas_classe = test_df[test_df['classe'] == c]     
    total_classe = len(linhas_classe)
    certos_classe = (linhas_classe['classe'] == linhas_classe['Previsto']).sum()
    taxa_acerto = certos_classe / total_classe 
    print(f"Classe {c}: ", taxa_acerto*100, "%")

Certos: 969 de 6191
Accuracy: 15.65%
Classe -1:  65.21739130434783 %
Classe 0:  92.9245283018868 %
Classe 1:  0.8695652173913043 %
